# Задание 2. Свертки и базовые слои 
Это задание будет являться духовным наследником первого. 
Вы уже научились делать шаги градиентного спуска и вспомнили, как устроен базовый линейный слой.
На этой неделе мы построим прототип базового фреймворка до конца (собственно, многое вы сможете скопировать, если захотите). 
Хоть вы уже и знаете о torch.nn, для выполнения задания его использовать нельзя. 
Однако все элементы, которые вы будете реализовывать, достаточно просты.

## Задача 1. (2 балла)
Реализуйте слой BatchNorm (nn.BatchNorm). 
## Задача 2. (2 балла)
Реализуйте слой Linear (nn.Linear). 
## Задача 3. (2 балла)
Реализуйте слой Dropout(nn.Dropout)

Вспомогательные классы

In [1]:
import torch
import math


class Module:
    """
    Базовый класс для всех слоев.
    """

    def __init__(self):
        self.training = True

    def forward(self, x):
        raise NotImplementedError

    def __call__(self, x):
        return self.forward(x)

    def train(self):
        """Переключает режим в training (обучение)."""
        self.training = True

    def eval(self):
        """Переключает режим в evaluation (инференс)."""
        self.training = False

    def parameters(self):
        """Возвращает список обучаемых параметров (тензоров с requires_grad=True)."""
        return []


class Sequential(Module):
    """
    Контейнер для последовательного выполнения слоев.
    """

    def __init__(self, *layers):
        super().__init__()
        self.layers = layers

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def train(self):
        super().train()
        for layer in self.layers:
            layer.train()

    def eval(self):
        super().eval()
        for layer in self.layers:
            layer.eval()

    def parameters(self):
        params = []
        for layer in self.layers:
            params.extend(layer.parameters())
        return params


Непосредственно слои

In [2]:
class BatchNorm(Module):
    """
    Слой Batch Normalization.
    """

    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum

        # Обучаемые параметры
        self.gamma = torch.ones(num_features, requires_grad=True)
        self.beta = torch.zeros(num_features, requires_grad=True)

        # Running stats (не обучаются градиентным спуском)
        self.running_mean = torch.zeros(num_features)
        self.running_var = torch.ones(num_features)

    def forward(self, x):
        if self.training:
            # Вычисляем статистики по батчу
            mean = x.mean(dim=0)
            # Для нормализации используем смещенную оценку дисперсии (как в PyTorch)
            var = x.var(dim=0, unbiased=False)

            # Обновляем running stats
            # Используем несмещенную оценку дисперсии для running_var (как в PyTorch)
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * x.var(dim=0, unbiased=True)

            x_hat = (x - mean) / torch.sqrt(var + self.eps)
        else:
            # Используем накопленные статистики
            x_hat = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)

        return self.gamma * x_hat + self.beta

    def parameters(self):
        return [self.gamma, self.beta]


class Linear(Module):
    """
    Полносвязный слой (Linear).
    y = xW + b
    """

    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        # Инициализация весов (аналогично PyTorch default initialization)
        stdv = 1. / math.sqrt(in_features)
        self.weights = torch.empty(in_features, out_features, requires_grad=True)
        self.weights.data.uniform_(-stdv, stdv)

        self.bias = torch.empty(out_features, requires_grad=True)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        return x @ self.weights + self.bias

    def parameters(self):
        return [self.weights, self.bias]


class Dropout(Module):
    """
    Слой Dropout.
    Во время обучения зануляет элементы входного тензора с вероятностью p.
    """

    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def forward(self, x):
        if self.training:
            # Генерируем маску из распределения Бернулли
            # 1 с вероятностью (1-p), 0 с вероятностью p
            mask = torch.empty_like(x).bernoulli_(1 - self.p)
            # Inverted Dropout: масштабируем, чтобы мат. ожидание не менялось
            return x * mask / (1 - self.p)
        return x


## Задача 4. {*} (2 балла, 1 за каждый следующий за слой)
Реализуйте одно или более из:
  - слой ReLU(nn.ReLU)
  - слой Sigmoid(nn.Sigmoid)
  - слой Softmax(nn.Softmax)

In [3]:
class ReLU(Module):
    """
    Слой активации ReLU.
    y = max(0, x)
    """

    def forward(self, x):
        return torch.max(x, torch.zeros_like(x))


class Sigmoid(Module):
    """
    Слой активации Sigmoid.
    y = 1 / (1 + exp(-x))
    """

    def forward(self, x):
        return 1 / (1 + torch.exp(-x))


class Softmax(Module):
    """
    Слой активации Softmax.
    """

    def forward(self, x):
        # Стабильный softmax: вычитаем максимум для избежания переполнения экспоненты
        x_max = x.max(dim=1, keepdim=True).values
        exp_x = torch.exp(x - x_max)
        return exp_x / exp_x.sum(dim=1, keepdim=True)



## Задача 5. {*}. 
Вы получите по 1 дополнительному баллу за слой, 
если реализуете в рамках фреймворка из задания 3 прошлой работы

Импорт из hw1:

In [4]:
class Optimizer:
    """Базовый класс для оптимизаторов."""

    def __init__(self, params, lr=0.01):
        self.params = list(params)
        self.lr = lr

    def step(self, grads):
        raise NotImplementedError


class Adam(Optimizer):
    """
    Adam (Adaptive Moment Estimation).
    Идея: Сочетает идеи Momentum и RMSProp. Использует оценки первого (mean) и второго (variance)
    моментов градиентов.
    m_t = beta1 * m_{t-1} + (1-beta1) * g_t
    v_t = beta2 * v_{t-1} + (1-beta2) * g_t^2
    Коррекция смещения (bias correction) для начальных шагов.
    """

    def __init__(self, params, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        super().__init__(params, lr)
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = [torch.zeros_like(p) for p in params]
        self.v = [torch.zeros_like(p) for p in params]
        self.t = 0

    def step(self, grads):
        self.t += 1
        for i, (param, grad) in enumerate(zip(self.params, grads)):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * grad
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * grad**2

            m_hat = self.m[i] / (1 - self.beta1**self.t)
            v_hat = self.v[i] / (1 - self.beta2**self.t)

            param -= self.lr * m_hat / (torch.sqrt(v_hat) + self.epsilon)


Все слои работают с оптимизатором

In [5]:
# Генерация синтетических данных
torch.manual_seed(42)
X = torch.randn(100, 10)
y = (torch.sum(X, dim=1) > 0).float().unsqueeze(1)  # Простая классификация

# Создание модели
model = Sequential(
	Linear(10, 20),
	BatchNorm(20),
	ReLU(),
	Dropout(0.2),
	Linear(20, 1),
	Sigmoid()
)

optimizer = Adam(model.parameters(), lr=0.01)

print("Model created.")

# Обучение
model.train()
epochs = 50

for epoch in range(epochs):
	# Forward
	predictions = model(X)

	# Loss (Binary Cross Entropy)
	loss = -torch.mean(y * torch.log(predictions + 1e-8) + (1 - y) * torch.log(1 - predictions + 1e-8))

	# Backward
	loss.backward()

	# Optimizer step
	grads = [p.grad for p in model.parameters()]
	with torch.no_grad():
		optimizer.step(grads)

	# Zero gradients
	for p in model.parameters():
		p.grad.zero_()

	if epoch % 10 == 0:
		print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

print(f"Final Loss: {loss.item():.4f}")

# Проверка eval режима
model.eval()
test_pred = model(X[:5])
print("Test predictions (first 5):", test_pred.detach().view(-1))
print("Test passed!")


Model created.
Epoch 0, Loss: 0.6788
Epoch 10, Loss: 0.4827
Epoch 20, Loss: 0.2554
Epoch 30, Loss: 0.1399
Epoch 40, Loss: 0.0711
Final Loss: 0.0503
Test predictions (first 5): tensor([8.7108e-01, 9.7752e-03, 1.0000e+00, 8.8701e-02, 6.7613e-05])
Test passed!
